In [ ]:
# ! pip install chromadb
import chromadb
from chromadb.config import Settings
from typing import List, Dict, Optional
import numpy as np
from pathlib import Path

import os

In [ ]:
class LawVectorDB:
    def __init__(self, vectordb_path : str = "DATA/vector_db", vectordb_name : str = "laws"):
        self.vectordb_path = Path(vectordb_path)
        self.vectordb_path.mkdir(parents = True, exist_ok = True)
        self.vectordb_name = vectordb_name

        """
        client - ChromaDB 클라이언트
        collection - ChromaDB 컬렉션 (문서 + 임베딩 + 메타데이터가 실제로 저장되는 공간)
        documents - 원본 문서 리스트 (List[Dict])
        """
        self.client = None
        self.collection = None
        self.search_engine = None
        self.documents = []

    def initialize_db(self):
        # client, collection, search_engine 초기화된 상태라면 불러오기
        if self.client is None:
            self.client = chromadb.PersistentClient(path = str(self.vectordb_path))
            self.collection = self.client.get_or_create_collection(self.vectordb_name)

            # 문서 수 확인
            doc_count = self.collection.count()
            print(f"총 {doc_count}개의 문서 발견")

    def add_documents(self, documents : List[Dict], embeddings : List[np.ndarray]):
        """ChromaDB에 문서 추가"""
        self.initialize_db()

        self.documents = documents

        try:
            # 배치 처리 - 100개씩
            batch_size = 100
            total_laws = len(documents)

            for i in range(0, total_laws, batch_size):
                end_idx = min(i + batch_size, total_laws)
                batch_laws = documents[i : end_idx]
                batch_embeddings = embeddings[i : end_idx]

                # 배치 길이 검증
                if len(batch_laws) != len(batch_embeddings):
                    raise ValueError(f"문서 수({len(batch_laws)})와 임베딩 수({len(batch_embeddings)})가 불일치합니다. i={i}, end_idx={end_idx}")

                print(f"배치 {i // batch_size + 1}/{(total_laws + batch_size - 1)//batch_size} : {end_idx - i}개 문서 저장 중...")

                ids = [f"doc_{j}" for j in range(i, end_idx)]
                texts = [doc.get("text", "") for doc in batch_laws]

                metadatas = []
                for doc in batch_laws:
                    metadata = {
                        "law_name" : str(doc.get("law_meta", {}).get("law_name", "")),
                        "eff_date" : str(doc.get("law_meta", {}).get("eff_date", "")),
                        "law_path" : str(doc.get("path", "")),
                        "section_type" : str(doc.get("section_type", "")),
                        "jomun_num" : str(doc.get("jomun_num", "")),
                        "hang_num" : str(doc.get("hang_no", ""))
                    }
                    metadatas.append(metadata)

                self.collection.add(
                    ids = ids,
                    documents = texts,
                    embeddings = [np.asarray(emb, dtype=np.float32).tolist() for emb in batch_embeddings],
                    metadatas = metadatas
                )

                print(f"문서 처리 : {end_idx} / {total_laws} 완료")

                # 메모리 압력 완화 - kernel이 자꾸 죽어요ㅜㅜ 
                del batch_laws
                del batch_embeddings
                del ids
                del texts
                del metadatas

        except Exception as e:
            print(f"문서 추가 중 오류 발생 : {e}")
            raise